In [1]:
# first we define the simulator and setup a quantum instance

from qiskit import Aer
from qiskit.utils import QuantumInstance

backend = Aer.get_backend('statevector_simulator')
qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

In [2]:
# then we setup our ansatz for the wavefunction

from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(2, reps=1)
ansatz.draw()

┌──────────────────────────────────────┐
q_0: ┤0                                     ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3]) │
q_1: ┤1                                     ├
     └──────────────────────────────────────┘

In [3]:
# now we have to define the observable we want the minimum eigenvalue of
from qiskit.opflow import X,Y,Z,I

observable = Z^I

In [4]:
# set-up the vqe algorithm, with an optimizer and wave function
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA

vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=qinstance)

In [5]:
# we could run this for many observables
result = vqe.compute_minimum_eigenvalue(observable)
print(result.eigenvalue)

(-0.9999999960195073+0j)


In [6]:
# now we have to define the observable we want the minimum eigenvalue of
from qiskit.opflow import X,Y,Z,I,H

observable = Y^I

result = vqe.compute_minimum_eigenvalue(observable)
print(result.eigenvalue)

0j


## Lets try it for the HO, should get eigenvalue 0

In [7]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *
from src.MatrixToPauliString import *

h_b_HO = 0.5*(p*p + m*m*q*q)
h_f_HO = -0.5*m#[bdag,b]

hamHO = Hamiltonian(h_b_HO, h_f_HO, {m:1, g:1},
                  2**1, standard_encode)

print(hamHO.pauliStrings)

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0


In [8]:
from src.qiskit_utilities import *

In [9]:
obsHOHAM = pauli_string_to_op(hamHO.pauliStrings)

In [10]:
result = vqe.compute_minimum_eigenvalue(obsHOHAM)
print(result.eigenvalue)

(8.098735786e-09+0j)


In [11]:
type(observable)

qiskit.opflow.primitive_ops.pauli_op.PauliOp

## DW

In [12]:
h_b_DW = 0.5*(p*p + (m*q + g*(q*q + mu*mu))**2)
h_f_DW = -0.5*(m+2*g*q)#[bdag,b]

hamDW = Hamiltonian(h_b_DW, h_f_DW, {m: 1, g:1, mu: 1},
                    2**1, standard_encode)

print(hamDW.pauliStrings)

3.625*I^0*I^1 + 0.5*I^0*Z^1 + 1.76776695296637*I^1*X^0 - 1.75*I^1*Z^0 + 0.707106781186548*X^0*Z^1


In [15]:
min(np.linalg.eig(getMatrix(hamDW.pauliStrings))[0])

(1.0786618070318874+0j)

In [17]:
from src.qiskit_utilities import *

obsDWHAM = pauli_string_to_op(hamDW.pauliStrings)

result = vqe.compute_minimum_eigenvalue(obsDWHAM)
print(result.eigenvalue)

(1.0786729066330358+0j)
